In [2]:
import random
import pandas
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.utils import check_array
from sklearn.cross_validation import train_test_split

import tensorflow as tf

import tensorflow.contrib.learn as skflow

train = pandas.read_csv('data/titanic_train.csv')
y, X = train['Survived'], train[['Age', 'SibSp', 'Fare']].fillna(0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

lr = LogisticRegression()
lr.fit(X_train, y_train)
print(accuracy_score(lr.predict(X_test), y_test))

0.664804469274


In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
train.Embarked.unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [15]:
y, X = train['Survived'], train[['Age', 'SibSp', 'Fare']].fillna(0)

X["male"] = (train.Sex == "male").astype(int)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

lr = LogisticRegression()
lr.fit(X_train, y_train)
print(accuracy_score(lr.predict(X_test), y_test))

0.782122905028


In [16]:
classifier = skflow.TensorFlowDNNClassifier(
    hidden_units=[20, 20, 20],
    n_classes=2,
    batch_size=32,
    steps=500
)

classifier.fit(X_train, y_train)

classifier, accuracy_score(classifier.predict(X_test), y_test)

Instructions for updating:
Pass `tf.contrib.learn.infer_real_valued_columns_from_input(x)` or `tf.contrib.learn.infer_real_valued_columns_from_input_fn(input_fn)` as `feature_columns`, where `x` or `input_fn` is your argument to `fit`, `evaluate`, or `predict`.
/home/yannick/bin/anaconda3/envs/tf/lib/python3.5/site-packages/tensorflow/python/ops/array_ops.py:1811: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  result_shape.insert(dim, 1)


(TensorFlowDNNClassifier(dropout=None, hidden_units=[20, 20, 20], batch_size=32, feature_columns=None, optimizer=None, steps=500),
 0.67039106145251393)

In [30]:
def dnn_tanh(X,y):
    layers = skflow.ops.dnn(X, [20, 60, 40, 20], tf.tanh)
    return skflow.models.logistic_regression(layers, y)

classifier = skflow.TensorFlowEstimator(
    model_fn=dnn_tanh,
    n_classes=2,
    batch_size=128,
    steps=500,
    learning_rate=0.055,
    verbose=2
)

%time classifier.fit(X_train, y_train)
classifier, accuracy_score(classifier.predict(X_test), y_test)

Instructions for updating:
Please use tf.contrib.layers.stack instead.
Instructions for updating:
Please use tf.contrib.layers.stack instead.


CPU times: user 6.16 s, sys: 300 ms, total: 6.46 s
Wall time: 5.01 s


(TensorFlowEstimator(clip_gradients=5.0, learning_rate=0.055, n_classes=2, continue_training=False, batch_size=128, params=None, optimizer=Adagrad, verbose=2, class_weight=None, steps=500),
 0.81564245810055869)

In [9]:
import random
from sklearn import datasets, cross_validation, metrics
import tensorflow as tf
from tensorflow.contrib import learn as skflow

random.seed(42)

# Load dataset and split it into train / test subsets.

digits = datasets.load_digits()
X = digits.images
y = digits.target

X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y,
    test_size=0.2, random_state=42)

# TensorFlow model using Scikit Flow ops

def conv_model(X, y):
    X = tf.expand_dims(X, 3)
    features = tf.reduce_max(skflow.ops.conv2d(X, 12, [3, 3]), [1, 2])
    features = tf.reshape(features, [-1, 12])
    features = skflow.ops.dnn(features, [200, 40], tf.tanh)
    return skflow.models.logistic_regression(features, y)

# Create a classifier, train and predict.
classifier = skflow.TensorFlowEstimator(model_fn=conv_model, n_classes=10,
                                        steps=500, learning_rate=0.05,
                                        batch_size=128)
classifier.fit(X_train, y_train)
score = metrics.accuracy_score(classifier.predict(X_test), y_test)
print('Accuracy: %f' % score)

Instructions for updating:
Please use tf.contrib.layers.conv2d instead.
Instructions for updating:
Please use tf.contrib.layers.stack instead.
Instructions for updating:
Please use tf.contrib.layers.conv2d instead.
Instructions for updating:
Please use tf.contrib.layers.stack instead.


Accuracy: 0.808333
